In [ ]:
import time as time
import pandas as pd
import os

from selenium import webdriver 
from selenium.webdriver.support.ui import Select

In [ ]:
# Sask physician home page
url = 'https://www.cps.sk.ca/imis/'

# Create a global dataframe to save our doctors
df = pd.DataFrame()

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument('-incognito')

browser = webdriver.Chrome(executable_path='../chromedriver', chrome_options=option)

In [ ]:
# Load the url
browser.get(url)

In [ ]:
# Grab the specialty drop down
select = Select(browser.find_element_by_id('ctl01_HomePageSearch_Search_ddlSpeciality'))

# Loop through each specialty
for i in range(1,len(select.options)):
    
    # regrab the specialty drop down when the page loads
    select = Select(browser.find_element_by_id('ctl01_HomePageSearch_Search_ddlSpeciality'))
    
    # Grab the ith specialty string value
    specialty = select.options[i].text
    
    # Select the ith specialty 
    select.select_by_index(i)
    
    # Click on the go button to search
    browser.find_element_by_id('ctl01_HomePageSearch_Search_Btn_Search').click()
    time.sleep(3)
    
    # Try and grab the doctor table if there are any doctors
    try:
        table = browser.find_element_by_xpath("//table[@class='table ng-table-responsive table-striped table-hover animated fadeIn']")
    except:
        # There are no doctors in the current specialty so we skip to the next specialty
        continue;
        
    # Grab all the table rows in the table 
    trList = table.find_elements_by_tag_name('tr')
    
    # Remove any headers
    trList = trList[1:]
    
    # For each row grab the relevant info
    for tr in trList:
        lastName = tr.find_elements_by_tag_name('td')[0].text
        firstName = tr.find_elements_by_tag_name('td')[1].text
        middleName = tr.find_elements_by_tag_name('td')[2].text
        city = tr.find_elements_by_tag_name('td')[3].text
        province = tr.find_elements_by_tag_name('td')[4].text
        country = tr.find_elements_by_tag_name('td')[5].text
        language = tr.find_elements_by_tag_name('td')[6].text
        postalCode = tr.find_elements_by_tag_name('td')[7].text
        phone = tr.find_elements_by_tag_name('td')[8].text
        fax  = tr.find_elements_by_tag_name('td')[9].text
        
        # Make a dict out of the info grabbed
        data = {'Last Name' : lastName, 
              'First Name' : firstName, 
              'Middle Name' : middleName, 
              'City' : city, 
              'Province' : province, 
              'Country' : country, 
              'Language' : language, 
              'Postal Code' : postalCode, 
              'Phone' : phone, 
              'Fax' : fax,
              'Specialty': specialty}
        
        # Append it to the data frame
        df = df.append(data, ignore_index=True)
        
        # Free the memory by deleting old variables 
        del lastName, firstName, middleName, city, province, country, language, postalCode, phone, fax

In [ ]:
df.to_csv('./csv/SK Physician.csv', index=False)